In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


I'm reading in the results of the combine customer billing script which is a combination of all the files in the customer billing folder

In [59]:
df = pd.read_csv('customer_billing.csv')

In [60]:
df.head(10)

,Billing Date,Parent ID,Customer ID,Service ID,Product ID,Product Description,Volume,Charge
0,2019-01-01,PARENT001,CUST955,SERV010,PROD058,Product 058,6,$243.75
1,2019-01-01,PARENT100,CUST957,SERV014,PROD058,Product 058,7,$160.00
2,2019-01-01,PARENT303,CUST958,SERV001,PROD058,Product 058,4,$208.75
3,2019-01-01,PARENT208,CUST959,SERV014,PROD058,Product 058,19,$226.25
4,2019-01-01,PARENT208,CUST962,SERV004,PROD058,Product 058,16,$226.25
5,2019-01-01,PARENT068,CUST963,SERV014,PROD058,Product 058,5,$143.75
6,2019-01-01,PARENT386,CUST964,SERV007,PROD058,Product 058,3,$183.75
7,2019-01-01,PARENT297,CUST965,SERV015,PROD058,Product 058,13,$115.00
8,2019-01-01,PARENT324,CUST966,SERV020,PROD058,Product 058,11,$211.25
9,2019-01-01,PARENT335,CUST967,SERV005,PROD058,Product 058,12,$100.00


In [61]:
df.describe(include='all')

,Billing Date,Parent ID,Customer ID,Service ID,Product ID,Product Description,Volume,Charge
count,2782955,2782955,2782955,2782955,2782955,2782955,2.782955e+06,2782955
unique,121,500,1000,20,50,50,NaN,1242
top,2020-01-01,PARENT472,CUST256,SERV003,PROD059,Product 059,NaN,$189.00
freq,23308,10183,3143,141962,60829,60829,NaN,7642
mean,NaN,NaN,NaN,NaN,NaN,NaN,9.997117e+00,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,5.481457e+00,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+00,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,5.000000e+00,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+01,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,1.500000e+01,NaN


In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2782955 entries, 0 to 2782954
Data columns (total 8 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   Billing Date         object
 1   Parent ID            object
 2   Customer ID          object
 3   Service ID           object
 4   Product ID           object
 5   Product Description  object
 6   Volume               int64 
 7   Charge               object
dtypes: int64(1), object(7)
memory usage: 169.9+ MB


This is an example of the type of dataset I'm constantly working with. It's a time-series dataset that consists of customer billing. One of the main tasks I have is to forecast customer volume and revenue.

In [63]:
# Convert 'Billing Date' to datetime
df['Billing Date'] = pd.to_datetime(df['Billing Date'])

In [64]:
# Convert 'Charge' to float
df['Charge'] = df['Charge'].str.replace('$', '').astype(float)

In [65]:
# Check to make sure the changes were made
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2782955 entries, 0 to 2782954
Data columns (total 8 columns):
 #   Column               Dtype         
---  ------               -----         
 0   Billing Date         datetime64[ns]
 1   Parent ID            object        
 2   Customer ID          object        
 3   Service ID           object        
 4   Product ID           object        
 5   Product Description  object        
 6   Volume               int64         
 7   Charge               float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 169.9+ MB


In [66]:
df.head()

,Billing Date,Parent ID,Customer ID,Service ID,Product ID,Product Description,Volume,Charge
0,2019-01-01,PARENT001,CUST955,SERV010,PROD058,Product 058,6,243.75
1,2019-01-01,PARENT100,CUST957,SERV014,PROD058,Product 058,7,160.00
2,2019-01-01,PARENT303,CUST958,SERV001,PROD058,Product 058,4,208.75
3,2019-01-01,PARENT208,CUST959,SERV014,PROD058,Product 058,19,226.25
4,2019-01-01,PARENT208,CUST962,SERV004,PROD058,Product 058,16,226.25


We want to predict at the parent level because that's how we like to view our customer base

In [67]:
# Aggregate the data to the Parent ID level
df_grouped = df.groupby(['Billing Date', 'Parent ID', 'Service ID', 'Product ID']).agg({'Volume': 'sum', 'Charge': 'sum'}).reset_index()

In [68]:
df_grouped.head()

,Billing Date,Parent ID,Service ID,Product ID,Volume,Charge
0,2014-01-01,PARENT001,SERV001,PROD002,17,82.0
1,2014-01-01,PARENT001,SERV001,PROD005,2,54.0
2,2014-01-01,PARENT001,SERV001,PROD007,6,56.0
3,2014-01-01,PARENT001,SERV001,PROD008,10,163.0
4,2014-01-01,PARENT001,SERV001,PROD011,19,163.0


In [69]:
df_grouped.shape

(2718874, 6)

In [70]:
from sklearn.preprocessing import MinMaxScaler

# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Select the columns to normalize
columns_to_normalize = ['Volume', 'Charge']

# Normalize the selected columns
df_grouped[columns_to_normalize] = scaler.fit_transform(df_grouped[columns_to_normalize])

In [71]:
# Make sure data is normalized
df_grouped.head()

,Billing Date,Parent ID,Service ID,Product ID,Volume,Charge
0,2014-01-01,PARENT001,SERV001,PROD002,0.275862,0.044568
1,2014-01-01,PARENT001,SERV001,PROD005,0.017241,0.005571
2,2014-01-01,PARENT001,SERV001,PROD007,0.086207,0.008357
3,2014-01-01,PARENT001,SERV001,PROD008,0.155172,0.157382
4,2014-01-01,PARENT001,SERV001,PROD011,0.310345,0.157382


In [72]:
# Ensure df_grouped is sorted by 'Billing Date' and Parent ID
df_grouped = df_grouped.sort_values(by=['Billing Date', 'Parent ID'])

# Feature Engineering

To perform feature engineering on the `df_grouped` dataframe, I wanted to consider the following:

1. Time-based Features:
    - Extract additional time-based features from the 'Billing Date' column, such as day of the week, day of the month, or hour of the day.

2. Rolling Window Statistics:
    - Compute rolling window statistics, such as rolling mean, rolling sum, or rolling standard deviation, for the 'Volume' and 'Charge' columns. This can provide insights into the trend and variability of the data over time.

3. Lagged Features:
    - Create lagged features by shifting the 'Volume' and 'Charge' columns by a certain number of time periods. This can capture the historical values of these variables and their impact on future values.

4. Categorical Encoding:
    - If there are categorical columns in the dataframe, such as 'Parent ID', 'Service ID', or 'Product ID', you can encode them using techniques like one-hot encoding or label encoding. This can enable the use of categorical variables in machine learning models.

5. Seasonality Features:
    - Identify and incorporate seasonality patterns in the data. For example, you can create binary variables indicating whether a particular month or quarter is associated with a specific season or holiday.

In [73]:
# Create columns for Year, Month, and Quarter of Billing Date
df_grouped['Year'] = df_grouped['Billing Date'].dt.year
df_grouped['Month'] = df_grouped['Billing Date'].dt.month
df_grouped['Quarter'] = df_grouped['Billing Date'].dt.quarter

In [74]:
# Make sure the columns were added
df_grouped.head()

,Billing Date,Parent ID,Service ID,Product ID,Volume,Charge,Year,Month,Quarter
0,2014-01-01,PARENT001,SERV001,PROD002,0.275862,0.044568,2014,1,1
1,2014-01-01,PARENT001,SERV001,PROD005,0.017241,0.005571,2014,1,1
2,2014-01-01,PARENT001,SERV001,PROD007,0.086207,0.008357,2014,1,1
3,2014-01-01,PARENT001,SERV001,PROD008,0.155172,0.157382,2014,1,1
4,2014-01-01,PARENT001,SERV001,PROD011,0.310345,0.157382,2014,1,1


In [75]:
# Create rolling averages for 'Volume' and 'Charge' for the past 3 months
df_grouped['Volume Rolling 3 Month Avg'] = df_grouped.groupby('Parent ID')['Volume'].transform(lambda x: x.rolling(3).mean())
df_grouped['Charge Rolling 3 Month Avg'] = df_grouped.groupby('Parent ID')['Charge'].transform(lambda x: x.rolling(3).mean())

In [76]:
# Calculate the difference between 'Volume' and 'Volume Rolling 3 Month Avg'
df_grouped['Volume Rolling 3 Month Avg Diff'] = df_grouped['Volume'] - df_grouped['Volume Rolling 3 Month Avg']

# Calculate the difference between 'Charge' and 'Charge Rolling 3 Month Avg'
df_grouped['Charge Rolling 3 Month Avg Diff'] = df_grouped['Charge'] - df_grouped['Charge Rolling 3 Month Avg']

In [77]:
# Make sure the calculations were made
df_grouped.head()

,Billing Date,Parent ID,Service ID,Product ID,Volume,Charge,Year,Month,Quarter,Volume Rolling 3 Month Avg,Charge Rolling 3 Month Avg,Volume Rolling 3 Month Avg Diff,Charge Rolling 3 Month Avg Diff
0,2014-01-01,PARENT001,SERV001,PROD002,0.275862,0.044568,2014,1,1,NaN,NaN,NaN,NaN
1,2014-01-01,PARENT001,SERV001,PROD005,0.017241,0.005571,2014,1,1,NaN,NaN,NaN,NaN
2,2014-01-01,PARENT001,SERV001,PROD007,0.086207,0.008357,2014,1,1,0.126437,0.019499,-0.040230,-0.011142
3,2014-01-01,PARENT001,SERV001,PROD008,0.155172,0.157382,2014,1,1,0.086207,0.057103,0.068966,0.100279
4,2014-01-01,PARENT001,SERV001,PROD011,0.310345,0.157382,2014,1,1,0.183908,0.107707,0.126437,0.049675


In [78]:
# Create a column to calculate the percentage change in 'Volume' from the previous month
df_grouped['Volume % Change'] = df_grouped.groupby('Parent ID')['Volume'].pct_change()

# Create a column to calculate the percentage change in 'Charge' from the previous month
df_grouped['Charge % Change'] = df_grouped.groupby('Parent ID')['Charge'].pct_change()

# Create a column to calculate the percentage change in 'Volume' from the previous quarter
df_grouped['Volume % Change Qtr'] = df_grouped.groupby('Parent ID')['Volume'].pct_change(periods=3)

# Create a column to calculate the percentage change in 'Charge' from the previous quarter
df_grouped['Charge % Change Qtr'] = df_grouped.groupby('Parent ID')['Charge'].pct_change(periods=3)

# Create a column to calculate the percentage change in 'Volume' from the previous year
df_grouped['Volume % Change Yr'] = df_grouped.groupby('Parent ID')['Volume'].pct_change(periods=12)

# Create a column to calculate the percentage change in 'Charge' from the previous year
df_grouped['Charge % Change Yr'] = df_grouped.groupby('Parent ID')['Charge'].pct_change(periods=12)

In [79]:
# Check out the new columns
df_grouped.head()

,Billing Date,Parent ID,Service ID,Product ID,Volume,Charge,Year,Month,Quarter,Volume Rolling 3 Month Avg,Charge Rolling 3 Month Avg,Volume Rolling 3 Month Avg Diff,Charge Rolling 3 Month Avg Diff,Volume % Change,Charge % Change,Volume % Change Qtr,Charge % Change Qtr,Volume % Change Yr,Charge % Change Yr
0,2014-01-01,PARENT001,SERV001,PROD002,0.275862,0.044568,2014,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-01-01,PARENT001,SERV001,PROD005,0.017241,0.005571,2014,1,1,NaN,NaN,NaN,NaN,-0.9375,-0.875000,NaN,NaN,NaN,NaN
2,2014-01-01,PARENT001,SERV001,PROD007,0.086207,0.008357,2014,1,1,0.126437,0.019499,-0.040230,-0.011142,4.0000,0.500000,NaN,NaN,NaN,NaN
3,2014-01-01,PARENT001,SERV001,PROD008,0.155172,0.157382,2014,1,1,0.086207,0.057103,0.068966,0.100279,0.8000,17.833333,-0.4375,2.53125,NaN,NaN
4,2014-01-01,PARENT001,SERV001,PROD011,0.310345,0.157382,2014,1,1,0.183908,0.107707,0.126437,0.049675,1.0000,0.000000,17.0000,27.25000,NaN,NaN


In [80]:
# Make a column for any months that fall in the holiday season
# November and December are considered the holiday season
df_grouped['Holiday Season'] = df_grouped['Month'].apply(lambda x: 1 if x in [11, 12] else 0)

In [81]:
# Show a sample of holiday season months
df_grouped[df_grouped['Holiday Season'] == 1].head()

,Billing Date,Parent ID,Service ID,Product ID,Volume,Charge,Year,Month,Quarter,Volume Rolling 3 Month Avg,Charge Rolling 3 Month Avg,Volume Rolling 3 Month Avg Diff,Charge Rolling 3 Month Avg Diff,Volume % Change,Charge % Change,Volume % Change Qtr,Charge % Change Qtr,Volume % Change Yr,Charge % Change Yr,Holiday Season
224788,2014-11-01,PARENT001,SERV002,PROD053,0.310345,0.147632,2014,11,4,0.224138,0.146240,0.086207,0.001393,0.384615,0.394737,0.000000,-0.226277,2.000000,-0.101695,1
224789,2014-11-01,PARENT001,SERV005,PROD052,0.310345,0.034819,2014,11,4,0.281609,0.096100,0.028736,-0.061281,0.000000,-0.764151,1.250000,-0.812030,8.000000,-0.390244,1
224790,2014-11-01,PARENT001,SERV007,PROD053,0.275862,0.115599,2014,11,4,0.298851,0.099350,-0.022989,0.016249,-0.111111,2.320000,0.230769,0.092105,4.333333,-0.194175,1
224791,2014-11-01,PARENT001,SERV007,PROD058,0.034483,0.164345,2014,11,4,0.206897,0.104921,-0.172414,0.059424,-0.875000,0.421687,-0.888889,0.113208,-0.714286,0.638889,1
224792,2014-11-01,PARENT001,SERV009,PROD002,0.189655,0.041783,2014,11,4,0.166667,0.107242,0.022989,-0.065460,4.500000,-0.745763,-0.388889,0.200000,10.000000,-0.062500,1


In [82]:
# Mark any months that fall into the covid-19 pandemic
# The pandemic started in March 2020
# We'll end the pandemic in December 2022
df_grouped['Covid-19 Pandemic'] = df_grouped['Billing Date'].apply(lambda x: 1 if x >= pd.Timestamp('2020-03-01') and x <= pd.Timestamp('2022-12-31') else 0)


In [83]:
# Show a sample of months during the pandemic
df_grouped[df_grouped['Covid-19 Pandemic'] == 1].sample(5)

,Billing Date,Parent ID,Service ID,Product ID,Volume,Charge,Year,Month,Quarter,Volume Rolling 3 Month Avg,...,Volume Rolling 3 Month Avg Diff,Charge Rolling 3 Month Avg Diff,Volume % Change,Charge % Change,Volume % Change Qtr,Charge % Change Qtr,Volume % Change Yr,Charge % Change Yr,Holiday Season,Covid-19 Pandemic
2320306,2022-08-01,PARENT132,SERV017,PROD033,0.034483,0.059053,2022,8,3,0.183908,...,-0.149425,-0.048747,-0.857143,-0.636364,-0.866667,-0.420765,-0.750,-0.602996,0,1
2143478,2021-12-01,PARENT201,SERV015,PROD044,0.172414,0.048816,2021,12,4,0.235632,...,-0.063218,-0.047632,-0.444444,-0.552077,0.000000,-0.212360,4.000,-0.825709,1,1
1734474,2020-06-01,PARENT075,SERV014,PROD015,0.275862,0.051671,2020,6,2,0.212644,...,0.063218,-0.003621,-0.058824,0.075362,7.000000,-0.476728,3.000,-0.658379,0,1
2288727,2022-06-01,PARENT423,SERV010,PROD031,0.224138,0.113649,2022,6,2,0.160920,...,0.063218,-0.014949,1.600000,-0.388306,0.000000,-0.375191,12.000,0.000000,0,1
1752955,2020-06-01,PARENT493,SERV008,PROD050,0.310345,0.133148,2020,6,2,0.137931,...,0.172414,-0.013278,17.000000,-0.075435,8.000000,3.192982,0.125,1.670391,0,1


In [84]:
# Create a column to calculate the number of months since the start of the pandemic
# Only consider months after the start of the pandemic  (March 2020)
df_grouped['Months Since Pandemic'] = (df_grouped['Billing Date'] - pd.Timestamp('2020-03-01')).dt.days / 30.44


In [85]:
# Show a sample of months since the start of the pandemic
df_grouped[df_grouped['Months Since Pandemic'] > 0].sample(5)

,Billing Date,Parent ID,Service ID,Product ID,Volume,Charge,Year,Month,Quarter,Volume Rolling 3 Month Avg,...,Charge Rolling 3 Month Avg Diff,Volume % Change,Charge % Change,Volume % Change Qtr,Charge % Change Qtr,Volume % Change Yr,Charge % Change Yr,Holiday Season,Covid-19 Pandemic,Months Since Pandemic
2316974,2022-08-01,PARENT060,SERV010,PROD021,0.241379,0.078552,2022,8,3,0.224138,...,-0.074095,0.400000,-0.110410,13.000000,-0.647500,-0.066667,-0.165680,0,1,29.007884
2302368,2022-07-01,PARENT230,SERV002,PROD007,0.344828,0.521170,2022,7,3,0.258621,...,0.186537,0.111111,4.016086,-0.090909,2.371171,0.428571,3.361305,0,1,27.989488
2643968,2023-10-01,PARENT336,SERV017,PROD059,0.051724,0.221170,2023,10,4,0.045977,...,0.011444,0.000000,1.211699,-0.750000,0.724213,-0.785714,0.028164,0,0,43.002628
2659359,2023-11-01,PARENT170,SERV013,PROD026,0.103448,0.037396,2023,11,4,0.189655,...,-0.018849,-0.500000,-0.574485,-0.647059,-0.819374,inf,-0.813929,1,0,44.021025
2633958,2023-10-01,PARENT113,SERV012,PROD041,0.189655,0.235306,2023,10,4,0.097701,...,0.008751,0.833333,0.378621,0.000000,-0.114053,0.833333,0.301618,0,0,43.002628


In [86]:
# Create dummy variables 'Year', 'Month', and 'Quarter'
df_grouped = pd.get_dummies(df_grouped, columns=['Year', 'Month', 'Quarter'], drop_first=True)

In [87]:
# Make sure the dummy variables were created
df_grouped.head()

,Billing Date,Parent ID,Service ID,Product ID,Volume,Charge,Volume Rolling 3 Month Avg,Charge Rolling 3 Month Avg,Volume Rolling 3 Month Avg Diff,Charge Rolling 3 Month Avg Diff,...,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,Quarter_2,Quarter_3,Quarter_4
0,2014-01-01,PARENT001,SERV001,PROD002,0.275862,0.044568,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
1,2014-01-01,PARENT001,SERV001,PROD005,0.017241,0.005571,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
2,2014-01-01,PARENT001,SERV001,PROD007,0.086207,0.008357,0.126437,0.019499,-0.040230,-0.011142,...,False,False,False,False,False,False,False,False,False,False
3,2014-01-01,PARENT001,SERV001,PROD008,0.155172,0.157382,0.086207,0.057103,0.068966,0.100279,...,False,False,False,False,False,False,False,False,False,False
4,2014-01-01,PARENT001,SERV001,PROD011,0.310345,0.157382,0.183908,0.107707,0.126437,0.049675,...,False,False,False,False,False,False,False,False,False,False


In [88]:
df_grouped.shape

(2718874, 43)

In [89]:
# Save the cleaned data to a new CSV file
df_grouped.to_csv('customer_billing_cleaned.csv', index=False)